<h3><b>Imports

In [17]:
from __future__ import print_function
import keras as kr
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
# A module implemented for a specific task of raeding MNIST raw files
import recognition.mnist_fread as mread

<h3><b>Variables

In [18]:
# CNN configuration variables
batch_size = 128
num_classes = 10
epochs = 12
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# MNIST files locations
loc = {'test_img': 'mnist/t10k-images.idx3-ubyte', 'test_lbl': 'mnist/t10k-labels.idx1-ubyte',
        'train_img': 'mnist/train-images.idx3-ubyte', 'train_lbl': 'mnist/train-labels.idx1-ubyte' }

<h3><b>Read MNIST data

In [19]:
train_img = mread.dig_images(loc['train_img'])
train_lbl = mread.labels(loc['train_lbl'])
train_lbl =  kr.utils.to_categorical(train_lbl, num_classes)

test_img = mread.dig_images(loc['test_img'])
test_lbl = mread.labels(loc['test_lbl'])
test_lbl = kr.utils.to_categorical(test_lbl, num_classes)

print('train_img shape: ', train_img.shape)
print('train_lbl shape: ', train_lbl.shape)

train_img shape:  (60000, 28, 28, 1)
train_lbl shape:  (60000, 10)


<h3><b>Build neural network

In [20]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
              activation='relu',
              input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=kr.losses.categorical_crossentropy,
            optimizer=kr.optimizers.Adadelta(),
            metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

<h3><b>Train model

In [13]:
model.fit(train_img, train_lbl,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_img, test_lbl))


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 153s 3ms/step - loss: 0.2958 - accuracy: 0.9093 - val_loss: 0.0590 - val_accuracy: 0.9805
Epoch 2/12
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0974 - accuracy: 0.9714 - val_loss: 0.0435 - val_accuracy: 0.9852
Epoch 3/12
60000/60000 [==============================] - 152s 3ms/step - loss: 0.0735 - accuracy: 0.9783 - val_loss: 0.0375 - val_accuracy: 0.9879
Epoch 4/12
60000/60000 [==============================] - 149s 2ms/step - loss: 0.0603 - accuracy: 0.9823 - val_loss: 0.0326 - val_accuracy: 0.9890
Epoch 5/12
60000/60000 [==============================] - 151s 3ms/step - loss: 0.0534 - accuracy: 0.9841 - val_loss: 0.0368 - val_accuracy: 0.9882
Epoch 6/12
60000/60000 [==============================] - 147s 2ms/step - loss: 0.0474 - accuracy: 0.9861 - val_loss: 0.0332 - val_accuracy: 0.9892
Epoch 7/12
60000/60000 [==============================] - 155

<h3><b>Loss and accuracy

In [15]:
score = model.evaluate(test_img, test_lbl, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.029013746584429463
Test accuracy: 0.9908999800682068


<h3><b>Save model

In [16]:
model.save('recognition/cnn_99.h5')